### Testing similarity between individuals

In [1]:
import random

In [2]:
def create_individual(effects, effect_structure):
    n_effects_chosen = random.randint(1, len(effects) - 2)
    selected_effects = random.sample(effects, n_effects_chosen)
    
    individ = {}
    for effect in selected_effects:
        if effect in effect_structure:
            structure = effect_structure[effect]
            individ[effect] = {
                param: round(random.uniform(range_[0], range_[1]), 2) 
                for param, (_, range_) in structure.items()
            }
    return individ

In [3]:
n_effects = 6
effects = [i for i in range(n_effects)]
effect_structure = {
    0: { "rate_hz": ('float', (0.0, 100.0)), },# Chorus
    1: { "delay_seconds": ('float', (0.0, 10.0)), },# Delay
    2: { "drive_db": ('float', (0.0, 50.0)), },# Distortion
    3: { "gain_db": ('float', (-50.0, 50.0)) },# Gain
    4: { "depth": ('float', (0.0, 1.0)), },# Phaser
    5: { "wet_level": ('float', (0.0, 1.0)), },# Reverb
}
effects_map = {
    0: 'Chorus',
    1: 'Delay',
    2: 'Distortion',
    3: 'Gain',
    4: 'Phaser',
    5: 'Reverb',
}

In [179]:
individ1 = create_individual(effects, effect_structure)
individ2 = create_individual(effects, effect_structure)

print(individ1)
print(individ2)

# Extract common keys
common_keys = individ1.keys() & individ2.keys()

similarity_value = 0.0
for key in common_keys:
    weight = min(1 / len(individ1), 1 / len(individ2))
    #print(f'Key: {key}, weight: {weight}')
    
    inner_dict = effect_structure[key]
    lower_v = None
    max_v = None
    for param, value in inner_dict.items():
        lower_v = value[1][0]
        max_v = value[1][1]
                
    param_name = list(effect_structure[key].keys())[0]
    r = individ1[key][param_name]
    x = individ2[key][param_name]
    
    #print("r = ", r)
    #print("x = ", x)
    #print("low limit = ", lower_v)
    #print("max limit = ", max_v)
    
    distance_to_lower_v = abs(r - lower_v)
    distance_to_max_v = abs(r - max_v)
    
    limit_chosen = None
    if distance_to_lower_v < distance_to_max_v:
        limit_chosen = max_v
    else:
        limit_chosen = lower_v
    
    #print("abs(x - r) = ", abs(x - r))
    #print("abs(r - limit_chosen) = ", abs(r - limit_chosen))
    
    obtained_value_normalized = 1 - abs(x - r)/abs(r - limit_chosen)
    similarity_value += weight * obtained_value_normalized
    
print(similarity_value)

{5: {'wet_level': 0.16}, 0: {'rate_hz': 64.92}, 2: {'drive_db': 34.74}, 3: {'gain_db': 16.62}}
{5: {'wet_level': 0.52}, 4: {'depth': 0.98}, 2: {'drive_db': 27.48}, 0: {'rate_hz': 70.46}}
0.5692779429490855


In [235]:
def similarity_individuals(individ1, individ2):
    common_keys = individ1.keys() & individ2.keys()
    similarity = 0.0
    for key in common_keys:
        weight = min(1 / len(individ1), 1 / len(individ2))        
        inner_dict = effect_structure[key]
        param_name, (range_name, (lower_v, max_v)) = next(iter(inner_dict.items()))
         
        param_name = list(effect_structure[key].keys())[0]
        r = individ1[key][param_name]
        x = individ2[key][param_name]
        
        limit_chosen = max_v if abs(r - lower_v) < abs(r - max_v) else lower_v
        
        obtained_value_normalized = 1 - abs(x - r)/abs(r - limit_chosen)
        similarity += weight * obtained_value_normalized
        
    return similarity

In [237]:
individ1 = create_individual(effects, effect_structure)
individ2 = create_individual(effects, effect_structure)
similarity = similarity_individuals(individ1, individ2)

print(individ1)
print(individ2)
print(similarity)

{4: {'depth': 0.41}}
{1: {'delay_seconds': 6.86}, 5: {'wet_level': 0.07}, 0: {'rate_hz': 93.18}}
0.0


### Testing similarity on Dataset

In [ ]:
import pandas as pd

df = pd.read_csv('./dataset_creation/dataset_audios.csv')
similarities = []
for index, row in df.iterrows():
    applied_effects = row['applied_effects']
    ga_effects = row['GA_effects']
    print(applied_effects)
    print(ga_effects)
    similarities.append(similarity_individuals(applied_effects, ga_effects))
    df['similarity_indv'] = pd.Series(similarities)
    df.to_csv('./dataset_creation/dataset_audios.csv', index=False)

In [1]:
import itertools
import numpy as np

threshold_values = np.arange(2, 3.5, 0.5)
fan_out_values = np.arange(10, 30, 10)
max_distance_atan_values = np.arange(90, 110, 10)
onset_threshold_values = np.arange(0.4, 0.8, 0.2)
frame_threshold_values = np.arange(0.4, 0.8, 0.2)
max_key_distance_values = np.arange(20, 60, 20)

param_combinations = list(itertools.product(
        threshold_values,
        fan_out_values,
        max_distance_atan_values,
        onset_threshold_values,
        frame_threshold_values,
        max_key_distance_values
))

print(len(param_combinations))

96


In [16]:
import pandas as pd

df = pd.read_csv('./results/dataset_model_test2.csv')
print(len(df.index))



192
